In [36]:
import asyncio
import threading
import time
from datetime import timedelta
import pandas as pd

In [37]:
def timer(start,end):
    elapsed = end - start
    return str(timedelta(seconds=elapsed))

In [38]:
async def calcEnergyDistance(key, data1, data2):
    LEN1 = len(data1)
    LEN2 = len(data2)
    
    A = 0
    for i in range(LEN1):
        for j in range(LEN2):
            A = A + abs(data1[i]-data2[j])
    A = 2*A/LEN1/LEN2
     
    B = 0
    for i in range(LEN1):
        for j in range(LEN1):
            B = B + abs(data1[i]-data1[j])
    B = B/LEN1/LEN1
     
    C = 0
    for i in range(LEN2):
        for j in range(LEN2):
            C = C + abs(data2[i]-data2[j])
    C = C/LEN2/LEN2
    
    Abs_Diff_Sum = 0
    for i in range(LEN1):
        for j in range(LEN2):
            Abs_Diff_Sum = Abs_Diff_Sum + abs(data1[i]-data2[j])
    ADS = 2*Abs_Diff_Sum/(LEN1*LEN2)
     
    results[key] = (A-B-C)/A/ADS

In [52]:
async def calcEnergyDistance2(key, data1, data2):
    LEN1 = len(data1)
    LEN2 = len(data2)
    
    A = 0
    B = 0
    Abs_Diff_Sum = 0
    for i in range(LEN1):
        for j in range(LEN2):
            A = A + abs(data1[i]-data2[j])
            Abs_Diff_Sum = Abs_Diff_Sum + abs(data1[i]-data2[j])
        for k in range(LEN1):
            B = B + abs(data1[i]-data1[k])
            
    A = 2*A/LEN1/LEN2
    B = B/LEN1/LEN1
    ADS = 2*Abs_Diff_Sum/(LEN1*LEN2)
     
    C = 0
    for i in range(LEN2):
        for j in range(LEN2):
            C = C + abs(data2[i]-data2[j])
    C = C/LEN2/LEN2
     
    results[key] = (A-B-C)/A/ADS

In [45]:
def async_between_callback(action):
    loop = asyncio.new_event_loop()
    asyncio.set_event_loop(loop)

    loop.run_until_complete(action)
    loop.close()

In [46]:
file1 = pd.read_csv("file1.csv")
file2 = pd.read_csv("file2.csv")

In [64]:
ITER = 400
NUM = 3
params = ['X001', 'X003', 'X008']
threads = []
results = {}

start = time.time()

for j in range(ITER):
    for i in range(NUM):
        param = params[i]
        data1 = file1[param].tolist()[:100]
        data2 = file2[param].tolist()[:100]
        key = "%s_%s" % (param, j)
        _thread = threading.Thread(target=async_between_callback, args=(calcEnergyDistance(key, data1, data2),))
        threads.append(_thread)
for _thread in threads:
    _thread.start()
for _thread in threads:
    _thread.join()
    
end = time.time()

print("Finish within " + timer(start, end))
# print(results)

Finish within 0:00:09.166951


In [63]:
ITER = 100
NUM = 3
params = ['X001', 'X003', 'X008']
threads = []
results = {}

start = time.time()

for j in range(ITER):
    for i in range(NUM):
        param = params[i]
        data1 = file1[param].tolist()[:100]
        data2 = file2[param].tolist()[:100]
        key = "%s_%s" % (param, j)
        _thread = threading.Thread(target=async_between_callback, args=(calcEnergyDistance2(key, data1, data2),))
        threads.append(_thread)
for _thread in threads:
    _thread.start()
for _thread in threads:
    _thread.join()
    
end = time.time()

print("Finish within " + timer(start, end))
# print(results)

Finish within 0:00:01.821830
